## JanataHack - E-Commerce Analytics ML Hackathon

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [30]:
train = pd.read_csv('train_8wry4cB.csv')
test = pd.read_csv('test_Yix80N0.csv')
sub = pd.read_csv('sample_submission_opxHi4g.csv')
#sub.drop(['gender'], axis=1, inplace=True)

train.shape, test.shape, sub.shape

((10500, 5), (4500, 4), (4500, 2))

In [31]:
#Data = Data.drop(['startTime','endTime'],axis=1)
#Data_formated = Data.set_index(Data.columns.drop('ProductList',1).tolist()).ProductList.str.split(';',expand = True).stack().reset_index().rename(columns={0:'ProductList'}).loc[:,Data.columns]
#data_train[['c1','c2','c3','c4','c5']] = Data_formated.ProductList.str.split('/',expand=True)
#Data_formated[['c1','c2','c3','c4','c5']] = data_train[['c1','c2','c3','c4','c5']]

In [32]:
#data['No_of_products'] = data['ProductList'].apply(lambda x: len(x.split(';')))
#data['A'] = data['ProductList'].apply(lambda x: x.split(';')[0].split('/')[0])
##data['B'] = data['ProductList'].apply(lambda x: x.split(';')[0].split('/')[1])
#data['C'] = data['ProductList'].apply(lambda x: x.split(';')[0].split('/')[2])
# start date without time
#data['sess_date'] = data['startTime'].apply(lambda x: x.split(' ')[0])

# start date and end date without time
#data['sess_start_date'] = data['startTime'].apply(lambda x: x.split(' ')[0])
#data['sess_end_date'] = data['endTime'].apply(lambda x: x.split(' ')[0])

# start and end time only
#data['sess_start_time'] = data['startTime'].apply(lambda x: x.split(' ')[1])
#data['sess_end_time'] = data['endTime'].apply(lambda x: x.split(' ')[1])

#data['sess_temp'] = data['sess_start_time'].apply(lambda x: time.strptime(x, "%H:%M"))

#data['sess_start_date'] = data['sess_start_date'].apply(lambda x: x.split('/')[0]+'/'+x.split('/')[1]+'/'+'20'+x.split('/')[2])
#data['sess_end_date'] = data['sess_end_date'].apply(lambda x: x.split('/')[0]+'/'+x.split('/')[1]+'/'+'20'+x.split('/')[2])

In [33]:
#def list_items(x):
#    items = []
#    for i in x.split(';'):
#        items.append(i.split('/')[-2])
#    return items
# To get the the D code which is most frequent, in case of more than one unique code
#def most_frequent(List): 
#    return max(set(List), key = List.count)

# This function to split the D function 
#def items_new_list(x):
#    new_items = []
#    for i in x:
#        new_items.append(i[:4])
#    return most_frequent(new_items)

#data['sess_items']=data['ProductList'].apply(lambda x: list_items(x))

# D code feature which is explained above.
#data['D'] = data['sess_items'].apply(lambda x: items_new_list(x))

In [34]:
train['startTime'] = pd.to_datetime(train['startTime'], format='%d/%m/%y %H:%M')
test['startTime'] = pd.to_datetime(test['startTime'], format='%d/%m/%y %H:%M')
train['endTime'] = pd.to_datetime(train['endTime'], format='%d/%m/%y %H:%M')
test['endTime'] = pd.to_datetime(test['endTime'], format='%d/%m/%y %H:%M')

train['Duration'] = train['endTime'] - train['startTime'] 
train['Duration'] = train['Duration'].astype('timedelta64[m]')

test['Duration'] = test['endTime'] - test['startTime'] 
test['Duration'] = test['Duration'].astype('timedelta64[m]')

train['ProductListCount'] = train['ProductList'].apply(lambda x: len(x.split(";")))
test['ProductListCount'] = test['ProductList'].apply(lambda x: len(x.split(";")))

In [35]:
train.head()

,session_id,startTime,endTime,ProductList,gender,Duration,ProductListCount
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female,1.0,4
1,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male,6.0,7
2,u19037,2014-12-01 15:58:00,2014-12-01 15:58:00,A00002/B00001/C00020/D16944/,female,0.0,1
3,u14556,2014-11-23 02:57:00,2014-11-23 03:00:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female,3.0,3
4,u24295,2014-12-17 16:44:00,2014-12-17 16:46:00,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male,2.0,2


In [36]:
def create_time_feats(df, col, drop=True):
    
    # Extracting date features
    df[col + '_day_month'] = df[col].dt.day
    df[col + '_day_year'] = df[col].dt.dayofyear
    df[col + '_day_week'] = df[col].dt.dayofweek
    
    if drop:
        df.drop([col], axis=1, inplace=True)
    return df

In [37]:
Data_train = train.set_index(train.columns.drop('ProductList',1).tolist()).ProductList.str.split(';',expand = True).stack().reset_index().rename(columns={0:'ProductList'}).loc[:,train.columns]
Data_test = test.set_index(test.columns.drop('ProductList',1).tolist()).ProductList.str.split(';',expand = True).stack().reset_index().rename(columns={0:'ProductList'}).loc[:,test.columns]

In [38]:
Data_train.head()

,session_id,startTime,endTime,ProductList,gender,Duration,ProductListCount
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/,female,1.0,4
1,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D02554/,female,1.0,4
2,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28436/,female,1.0,4
3,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28437/,female,1.0,4
4,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/,male,6.0,7


In [39]:
Data_train[['category_code','sub_category_code','sub_sub_category_code','product_code','c5']] = Data_train.ProductList.str.split('/',expand=True)
Data_test[['category_code','sub_category_code','sub_sub_category_code','product_code','c5']] = Data_test.ProductList.str.split('/',expand=True)

In [40]:
Data_train.head()

,session_id,startTime,endTime,ProductList,gender,Duration,ProductListCount,category_code,sub_category_code,sub_sub_category_code,product_code,c5
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/,female,1.0,4,A00002,B00003,C00006,D28435,
1,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D02554/,female,1.0,4,A00002,B00003,C00006,D02554,
2,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28436/,female,1.0,4,A00002,B00003,C00006,D28436,
3,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28437/,female,1.0,4,A00002,B00003,C00006,D28437,
4,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/,male,6.0,7,A00001,B00009,C00031,D29404,


In [41]:
#Data_train = create_time_feats(Data_train, 'startTime', drop=True)
#Data_train = create_time_feats(Data_train, 'endTime', drop=True)
#Data_test = create_time_feats(Data_test, 'startTime', drop=True)
#Data_test = create_time_feats(Data_test, 'endTime', drop=True)

In [42]:
Data_train.head()

,session_id,startTime,endTime,ProductList,gender,Duration,ProductListCount,category_code,sub_category_code,sub_sub_category_code,product_code,c5
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/,female,1.0,4,A00002,B00003,C00006,D28435,
1,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D02554/,female,1.0,4,A00002,B00003,C00006,D02554,
2,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28436/,female,1.0,4,A00002,B00003,C00006,D28436,
3,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28437/,female,1.0,4,A00002,B00003,C00006,D28437,
4,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/,male,6.0,7,A00001,B00009,C00031,D29404,


In [43]:
Data_train = Data_train.drop(['ProductList','c5','startTime','endTime'],axis=1)
Data_test = Data_test.drop(['ProductList','c5','startTime','endTime'],axis=1)

In [44]:
Data_train[list(Data_train.columns)].dtypes

session_id                object
gender                    object
Duration                 float64
ProductListCount           int64
category_code             object
sub_category_code         object
sub_sub_category_code     object
product_code              object
dtype: object

In [45]:
features = list(set(Data_train.columns) - set(['gender']))

In [46]:
target= 'gender'

In [47]:
Data_train[features].dtypes

session_id                object
Duration                 float64
ProductListCount           int64
sub_category_code         object
sub_sub_category_code     object
product_code              object
category_code             object
dtype: object

In [48]:
from sklearn.preprocessing import LabelEncoder
le_target = LabelEncoder()
le_session = LabelEncoder()
Data_train[target] = le_target.fit_transform(Data_train[target])
df = pd.concat([Data_train,Data_test])
df['session_id'] = le_session.fit_transform(df['session_id'])
cat_features = list(set(features)-set([target,'session_id']))
for i in cat_features:
    df[i] = LabelEncoder().fit_transform(df[i])

In [49]:
df.head()

,Duration,ProductListCount,category_code,gender,product_code,session_id,sub_category_code,sub_sub_category_code
0,1,3,1,0.0,17574,6158,2,5
1,1,3,1,0.0,1789,6158,2,5
2,1,3,1,0.0,17575,6158,2,5
3,1,3,1,0.0,17576,6158,2,5
4,6,6,0,1.0,18146,252,8,30


In [50]:
Data_train = df[df[target].notnull()]
Data_test =  df[df[target].isnull()]

In [51]:
del df

In [52]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,VotingClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier,ExtraTreesClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import xgboost as xgb
#import lightgbm as lgb
#import catboost as cat
def baseliner(train, features, target, cv=3, metric='accuracy'):
    """
    Function for baselining Models which return CV Score, Train Score, Valid Score
    """
    print("Baseliner Models\n")
    eval_dict = {}
    models = [xgb.XGBClassifier(), GradientBoostingClassifier(), LogisticRegression(), 
              RandomForestClassifier(), DecisionTreeClassifier(), AdaBoostClassifier(),ExtraTreeClassifier(),ExtraTreesClassifier(),
              KNeighborsClassifier(),BaggingClassifier()
             ]
    print("Model Name \t |   CV")
    print("--" * 50)

    for index, model in enumerate(models, 0):
        model_name = str(model).split("(")[0]
        eval_dict[model_name] = {}

        results = cross_val_score(model, train[features], train[target], cv=cv, scoring=metric)
        eval_dict[model_name]['cv'] = results.mean()

        print("%s \t | %.4f \t" % (
            model_name[:12], eval_dict[model_name]['cv']))

In [53]:
baseliner(Data_train,features,target)

Baseliner Models

Model Name 	 |   CV
----------------------------------------------------------------------------------------------------


C:\Users\braja.bera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\braja.bera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\braja.bera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


XGBClassifie 	 | 0.8874 	
GradientBoos 	 | 0.8880 	
LogisticRegr 	 | 0.8291 	
RandomForest 	 | 0.8908 	
DecisionTree 	 | 0.8635 	
AdaBoostClas 	 | 0.8842 	
ExtraTreeCla 	 | 0.8202 	
ExtraTreesCl 	 | 0.8807 	
KNeighborsCl 	 | 0.7879 	
BaggingClass 	 | 0.9007 	


In [54]:
BGC = BaggingClassifier()

In [55]:
BGC.fit(Data_train[features],Data_train[target])
Data_pred = BGC.predict(Data_test[features])
Data_test['gender'] = Data_pred
Data_test['gender'] = Data_test['gender'].astype(int)

In [56]:
Data_test.head()

,Duration,ProductListCount,category_code,gender,product_code,session_id,sub_category_code,sub_sub_category_code
0,0,0,1,0,12675,2111,2,5
1,0,0,1,0,1407,9724,4,66
2,0,0,1,0,8129,1794,1,3
3,3,3,1,1,14345,12638,2,77
4,3,3,1,1,14346,12638,2,77


In [58]:
Data_test['gender'] = le_target.inverse_transform(Data_test['gender'])
Data_test['session_id'] = le_session.inverse_transform(Data_test['session_id'])

C:\Users\braja.bera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\braja.bera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [62]:
Data_final_sub = Data_test[['session_id','gender']].drop_duplicates()

In [67]:
Data_final_sub.to_csv("xgb_model_final_submisison.csv",index=False)